In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')
snorkel_postgres = os.environ.get('SNORKELDB','').startswith('postgres')
print snorkel_postgres

True


### Parse

In [2]:
# If necessary:
import os
if snorkel_postgres:
    os.environ['SNORKELDBNAME'] = 'parts_regex'
    os.system("dropdb " + os.environ['SNORKELDBNAME'])
    os.system("createdb " + os.environ['SNORKELDBNAME'])
else:
    try:
        os.remove('snorkel.db')
    except:
        pass

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
import os

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/html/'
pdf_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/pdf/'

if snorkel_postgres:
    from snorkel.async_parser import parse_corpus, HTMLParser, AsyncOmniParser
   
    # PARSE DEV
    print "Starting async parse..."
    doc_parser = HTMLParser()
    context_parser = AsyncOmniParser(blacklist=['style'], flatten=['span','br'], 
                                     tabular=True, lingual=True,
                                     visual=True, pdf_path=pdf_path)
    %time corpus = parse_corpus(session, 'Hardware Dev', docs_path,\
                                doc_parser, context_parser,\
                                max_docs=125, parallel=3)
    
else:
    from snorkel.parser import CorpusParser, HTMLParser, OmniParser
    from snorkel.utils import get_ORM_instance

    # PARSE DEV
    print "Starting sync parse..."
    doc_parser = HTMLParser(path=docs_path)
    context_parser = OmniParser(blacklist=['style'], flatten=['span','br'], 
                                tabular=True, lingual=True,
                                visual=True, pdf_path=pdf_path)
    cp = CorpusParser(doc_parser, context_parser, max_docs=125)
    %time corpus = cp.parse_corpus(name='Hardware Dev', session=session)

    session.add(corpus)
    session.commit()

print "%s contains %d documents" % (corpus, len(corpus))

Starting async parse...
[========================================] 100%
CPU times: user 242 ms, sys: 82.8 ms, total: 325 ms
Wall time: 3min 25s
Corpus (Hardware Dev) contains 122 documents


In [4]:
# If necessary:
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ corpus');

### Load

In [5]:
# from snorkel.models import Corpus
# import os

# os.system('cp snorkel.db\ corpus snorkel.db');
# from snorkel import SnorkelSession
# session = SnorkelSession()

# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Dev').one()
# print "%s contains %d Documents" % (corpus, len(corpus))

### Extraction

In [6]:
from snorkel.models import candidate_subclass
from hardware_spaces import get_space
from hardware_matchers import get_matcher
from hardware_throttlers import get_part_throttler_wrapper

Part = candidate_subclass('Part', ['part'])
part_ngrams = get_space('part')
part_matcher = get_matcher('part')
part_throttler = get_part_throttler_wrapper()
# part_throttler = None

In [7]:
# from hardware_matchers import get_matcher

# part_matcher = get_matcher('part')
# part_matcher.rgx

In [8]:
# Attempt at alternative regex:
# from snorkel.matchers import RegexMatchSpan, Inverse, Intersect
# long_enough = RegexMatchSpan(rgx='[A-Z2][A-Z0-9\-]{4,}(?:\/(?:DG|HF))?', ignore_case=False)
# at_least_one_number = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[0-9][A-Z0-9\-\/]*', ignore_case=False)
# at_least_one_letter = RegexMatchSpan(rgx='[A-Z0-9\-\/]*[A-Z][A-Z0-9\-\/]*', ignore_case=False) 
# bad_form = Inverse(RegexMatchSpan(rgx='(?:[A-Z]+\d)|(?:[A-Z0-9\-\/]{0,2}\-[A-Z0-9\-\/]*)'))
# bad_start = Inverse(RegexMatchSpan(rgx='(?:T[O0]|SOT|BRD|FIGURE|EHP)[A-Z0-9\-\/]*'))
# bad_end = Inverse(RegexMatchSpan(rgx='[A-Z0-9\-\/]*(TYP|MAX|MIN|BASE|HZ|MM|VDC|VDF)'))
# part_matcher = Intersect(long_enough, at_least_one_number, at_least_one_letter, 
#                          bad_form, bad_start, bad_end)

In [9]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, 
                        [part_ngrams], 
                        [part_matcher],
                        throttler=part_throttler)

In [10]:
%time candidates = ce.extract(corpus.documents, 'Hardware Dev Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%
CPU times: user 2min 23s, sys: 3.09 s, total: 2min 26s
Wall time: 5min
Candidate Set (Hardware Dev Candidates) contains 3179 Candidates


In [11]:
# If necessary:
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db snorkel.db\ candidates');

### Second Pass

In [12]:
# If necessary:
if not snorkel_postgres:
    import os
    os.system('cp snorkel.db\ candidates snorkel.db');

In [13]:
# from hardware_utils import get_gold_parts_by_doc, get_manual_parts_by_doc
# from snorkel.utils import get_ORM_instance
# from snorkel.models import Corpus

# corpus = get_ORM_instance(Corpus, session, 'Hardware Dev')

# # parts_by_doc = get_gold_parts_by_doc()
# parts_by_doc = get_manual_parts_by_doc(corpus)
# # parts_by_doc = None

In [14]:
# import cPickle as pickle
# pickle_file = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/parts_by_doc.pkl'

# with open(pickle_file, 'w') as f:
#     pickle.dump(parts_by_doc, f)

In [15]:
import cPickle as pickle
pickle_file = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/parts_by_doc.pkl'
with open(pickle_file, 'r') as f:
    parts_by_doc = pickle.load(f)

### Evaluation

In [16]:
from hardware_utils import get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/dev/hardware_dev_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
print "%d gold parts loaded" % len(gold_parts)

927 gold parts loaded


In [17]:
from hardware_utils import parts_f1

%time TP, FP, FN = parts_f1(candidates, gold_parts, parts_by_doc)

Scoring on Entity-Level Gold Data
Corpus Precision 0.826
Corpus Recall    0.923
Corpus F1        0.872
----------------------------------------
TP: 856 | FP: 180 | FN: 71

CPU times: user 13.6 s, sys: 602 ms, total: 14.2 s
Wall time: 17.2 s


In [18]:
from pprint import pprint
pprint(FP)

[(u'2N3906-D', u'2N3906/D'),
 (u'BC337', u'BC33716'),
 (u'BC337', u'BC33725'),
 (u'BC337', u'BC33740'),
 (u'BC337', u'BC33740BU'),
 (u'BC337', u'BC338-16'),
 (u'BC337', u'BC338-40'),
 (u'BC337', u'BC33825'),
 (u'BC337-D', u'BC337-XX'),
 (u'BC546', u'BC548A'),
 (u'BC546', u'BC549A'),
 (u'BC546', u'BC550A'),
 (u'BC546', u'BC550B'),
 (u'BC546-BC548C(TO-92)', u'BC546-BC548C'),
 (u'BC546-BC548C(TO-92)', u'BC547A/B/C'),
 (u'BC546-BC548C(TO-92)', u'BC548A/B/C'),
 (u'BC546A_SERIES_B14-521026', u'BC546A/B/C'),
 (u'BC546A_SERIES_B14-521026', u'BC550A/B/C'),
 (u'BC546_DIOTEC', u'BC546C'),
 (u'BC546_DIOTEC', u'BC549A'),
 (u'BC547', u'BC548A'),
 (u'BC547', u'BC548BU'),
 (u'BC547', u'BC549A'),
 (u'BC547', u'BC550A'),
 (u'BC547', u'BC550B'),
 (u'BC818-40LT1-D', u'BC818-40LT1'),
 (u'CSEMS05382-1', u'BC546-48ABC'),
 (u'DIODESINCORPORATED_FCX491ATA', u'AP02001'),
 (u'DIODESINCORPORATED_FCX491ATA', u'AP02002'),
 (u'DIODESINCORPORATED_FZT651TC', u'AP02001'),
 (u'DIODESINCORPORATED_FZT651TC', u'AP02002'),


In [19]:
pprint(FN)

[('CENTRALSEMICONDUCTORCORP_CENU45', 'CEN-U45'),
 ('CENTRALSEMICONDUCTORCORP_CMPT5401ETR', 'CMPT5401E'),
 ('CENTRALSEMICONDUCTORCORP_CXT4033TR', 'CXT4033'),
 ('DIODESINCORPORATED_2DD26527', '2DD2652'),
 ('DIODESINCORPORATED_2DD26527', '2DD2652-7'),
 ('DIODESINCORPORATED_FCX491ATA', 'FCX491A'),
 ('DIODESINCORPORATED_ZXT690BKTC', 'ZXT690BKQTC'),
 ('DIODESINCORPORATED_ZXT690BKTC', 'ZXT690BKTC'),
 ('DISES00490-1', 'BC846AW'),
 ('DISES00490-1', 'BC846BW'),
 ('DISES00490-1', 'BC847AW'),
 ('DISES00490-1', 'BC847BW'),
 ('DISES00490-1', 'BC847CW'),
 ('DISES00490-1', 'BC848AW'),
 ('DISES00490-1', 'BC848BW'),
 ('DISES00490-1', 'BC848CW'),
 ('DISES00490-1', 'BC849BW'),
 ('DISES00490-1', 'BC849CW'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310O'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310R'),
 ('FAIRCHILDSEMICONDUCTOR_KSC2310YTA', 'KSC2310Y'),
 ('LITES00689-1', 'BC337-25'),
 ('MICROCOMMERCIALCO_2N3904AP', '2N3904-AP'),
 ('MICROCOMMERCIALCO

In [21]:
from hardware_utils import entity_to_candidates
from snorkel.lf_helpers import *

entity = FP[0]
print entity
print

matches = entity_to_candidates(entity, candidates)
print "# Matches: %d" % len(matches)
print

(u'2N3906-D', u'2N3906/D')

# Matches: 1



In [ ]:
from hardware_utils import print_table_info

candidate = matches[1]
# print candidate
# print
print_table_info(candidate.part)
# throttler = get_part_throttler()
# print throttler(candidate)

In [ ]:
c = candidates[100]
print c
print c.part.get_attrib_tokens('top'), c.part.get_attrib_tokens('bottom')
from snorkel.lf_helpers import *
print get_min_col_num(c)
print get_max_col_num(c)
print list(get_row_ngrams(c))
print_table_info(c.part)
# print list(get_vert_aligned_ngrams(c.part))

In [ ]:
# from snorkel.lf_helpers import *
# c = candidates[110]
# print c
# get_prev_sibling_tags(c.part).count('p')

The End.